In [3]:
#load and clean the Flight data
import pandas as pd
import numpy as np

flt=pd.read_csv("https://raw.githubusercontent.com/Messimx/Driven_Supply_Chain_Project01/master/Flight.csv")
htl=pd.read_csv("https://raw.githubusercontent.com/Messimx/Driven_Supply_Chain_Project01/master/Hotel.csv")
flt['wk_min']=flt.iloc[:,2:5].min(1)
flt['wk_name']=flt.iloc[:,2:5].idxmin(1)
flt['wkn_min']=flt.iloc[:,5:8].min(1)
flt['wkn_name']=flt.iloc[:,5:8].idxmin(1)
flt.head()

,Departure,Arrival,Expedia_wk,Kayak_wk,CheapOAir_wk,Expedia_wkn,Kayak_wkn,CheapOAir_wkn,wk_min,wk_name,wkn_min,wkn_name
0,NY(JFK),ABQ,120.0,161,144.8,136,148,144.8,120.0,Expedia_wk,136.0,Expedia_wkn
1,NY(JFK),CLE,113.0,105,119.8,113,105,144.3,105.0,Kayak_wk,105.0,Kayak_wkn
2,NY(JFK),MIA,76.2,80,84.2,131,123,111.2,76.2,Expedia_wk,111.2,CheapOAir_wkn
3,NY(JFK),SEA,131.0,137,167.8,156,149,164.8,131.0,Expedia_wk,149.0,Kayak_wkn
4,ABQ,NY(JFK),160.0,121,151.8,219,121,151.8,121.0,Kayak_wk,121.0,Kayak_wkn


In [4]:
#load and clean the Hotel data
htl['wk_min']=htl.iloc[:,[1,3]].min(1)
htl['wkn_min']=htl.iloc[:,[5,7]].min(1)
htl['wk_web']=htl.iloc[:,[1,3]].idxmin(1)
htl['wkn_web']=htl.iloc[:,[5,7]].idxmin(1)
htl.loc[:,'wk_web']=htl.loc[:,'wk_web'].replace('price','name',regex=True)
htl.loc[:,'wkn_web']=htl.loc[:,'wkn_web'].replace('price','name',regex=True)
for i in range(0,len(htl['wk_web'])):
    htl.at[i,'wk_name']=htl.at[i,htl.at[i,'wk_web']]
    htl.at[i,'wkn_name']=htl.at[i,htl.at[i,'wkn_web']]
htl=htl.drop(htl.columns[1:9], axis=1)
htl

,Location,wk_min,wkn_min,wk_web,wkn_web,wk_name,wkn_name
0,NY(JFK),279,224,EKT_wk_name,EKT_wkn_name,Hilton New York JFK Airport,SAVE 12%\nHilton New York JFK Airport\n
1,ABQ,127,111,CheapOAir_wk_name,CheapOAir_wkn_name,Doubletree By Hilton Albuquerque,Doubletree By Hilton Albuquerque
2,CLE,149,159,EKT_wk_name,EKT_wkn_name,Hampton Inn & Suites Cleveland-Airport,Hilton Garden Inn Cleveland Airport
3,MIA,159,149,CheapOAir_wk_name,CheapOAir_wkn_name,Hilton Miami Airport,Hilton Miami Airport
4,SEA,159,139,EKT_wk_name,EKT_wkn_name,Hilton Seattle Airport & Conference Center,Hilton Seattle Airport & Conference Center


In [5]:
#Build the basic data frame of resutls
import itertools
import math

df=pd.DataFrame(columns=['city','persons','start_date','wk','wkn','hotel','hotel_name','flight','hotel_cost','meal_cost','flight_cost','transportation_cost','cumulative_cost'])
cities=['NY(JFK)','ABQ','CLE','MIA','SEA']
sub_cities=list(cities)
sub_cities.remove('NY(JFK)')

gen = itertools.permutations(cities)
citi=[list(i) for i in gen]
citi=sum(citi,[])

i=0
j=0
while 1==1:
    cities=['NY(JFK)','ABQ','CLE','MIA','SEA']
    df.at[i,'wkn']=0
    if (i%6==5):
        cities.remove(df.at[i-1,'city'])
        df.at[i,'city']=cities
    else:
        df.at[i,'city']=citi[j]
        j+=1
    i+=1
    if(j%5==0):
        if (citi[j]!='NY(JFK)'):
            df.at[i,'wkn']=0
            cities.remove(df.at[i-1,'city'])
            df.at[i,'city']=cities
            break
#Count the number of persons for each city
i=0
for row in range(0,int(df.shape[0]/6)):
    for j in range(0,6):
        if(j==0):
            df.at[i+j,'persons']=1
        elif(j==5):
            df.at[i+j,'persons']=0
        else:
            df.at[i+j,'persons']=df.at[i+j-1,'persons']+1
    i+=6

#Calculate the start date, flight website, flight cost in each city
i=0
for row in range(0,int(df.shape[0]/6)):
    for j in range(0,6):
        if(j==0):
            df.at[i+j,'start_date']=1
            df.at[i+j,'flight_cost']=0
            df.at[i+j,'transportation_cost']=0
            df.at[i+j,'flight']='Start'
        else:
            if(df.at[i+j-1,'city']=='NY(JFK)'):
                df.at[i+j,'start_date']=df.at[i+j-1,'start_date']+3
            elif(df.at[i+j-1,'city']=='ABQ'):
                df.at[i+j,'start_date']=df.at[i+j-1,'start_date']+2
            elif(df.at[i+j-1,'city']=='CLE'):
                df.at[i+j,'start_date']=df.at[i+j-1,'start_date']+2
            elif(df.at[i+j-1,'city']=='MIA'):
                df.at[i+j,'start_date']=df.at[i+j-1,'start_date']+3
            else:
                df.at[i+j,'start_date']=df.at[i+j-1,'start_date']+3
            if(j!=5):
                rnum_flt=flt.loc[flt['Departure']==df.at[i+j-1,'city']].loc[flt['Arrival']==df.at[i+j,'city']].index[0]
                if((df.at[i+j,'start_date']%7>1) and (df.at[i+j,'start_date']%7<5)):
                    df.at[i+j,'flight_cost']=flt.at[rnum_flt,'wk_min']*df.at[i+j-1,'persons']
                    df.at[i+j,'flight']=flt.at[rnum_flt,'wk_name']
                else:
                    df.at[i+j,'flight_cost']=flt.at[rnum_flt,'wkn_min']*df.at[i+j-1,'persons']
                    df.at[i+j,'flight']=flt.at[rnum_flt,'wkn_name']
            else:
                if((df.at[i+j,'start_date']%7>1) and (df.at[i+j,'start_date']%7<5)):
                    df.at[i+j,'flight_cost']=flt.loc[flt['Departure']==df.at[i+j-1,'city'],['wk_min']].sum()[0]
                    df.at[i+j,'flight']=flt.loc[flt['Departure']==df.at[i+j-1,'city'],['wk_name']].values.tolist()
                else:
                    df.at[i+j,'flight_cost']=flt.loc[flt['Departure']==df.at[i+j-1,'city'],['wkn_min']].sum()[0]
                    df.at[i+j,'flight']=flt.loc[flt['Departure']==df.at[i+j-1,'city'],['wkn_name']].values.tolist()
            for wkn in range(df.at[i+j-1,'start_date']+1,df.at[i+j,'start_date']):
                if (wkn%7==6 or wkn%7==0):
                    df.at[i+j,'start_date']+=1
                    df.at[i+j,'wkn']=1
                    break
    i+=6
    
#Decide how many week days and weekend days they stay in each city
i=0
for row in range(0,int(df.shape[0]/6)):
    for j in range(0,6):
        if (j!=5):
            stay=df.at[i+j+1,'start_date']-df.at[i+j,'start_date']
            wk=0
            wkn=0
            for s in range(0,stay):
                if(((df.at[i+j,'start_date']+s)%7>1) and ((df.at[i+j,'start_date']+s)%7<5)):
                    wk+=1
                else:
                    wkn+=1
            df.at[i+j,'wk']=wk
            df.at[i+j,'wkn']=wkn
        else:
            df.at[i+j,'wk']=1-df.at[i+j,'wkn']
    i+=6
#Calculate the hotel cost in each city
i=0
for row in range(0,int(df.shape[0]/6)):
    for j in range(0,6):
        if (j!=5):
            rnum_htl=htl.loc[htl['Location']==df.at[i+j,'city']].index[0]
            df.at[i+j,'hotel_cost']=(htl.at[rnum_htl,'wk_min']*df.at[i+j,'wk']+htl.at[rnum_htl,'wkn_min']*df.at[i+j,'wkn'])*df.at[i+j,'persons']
            if(df.at[i+j,'wkn']==0):
                df.at[i+j,'hotel']=htl.at[rnum_htl,'wk_web']
                df.at[i+j,'hotel_name']=htl.at[rnum_htl,'wk_name']
            elif(df.at[i+j,'wk']==0):
                df.at[i+j,'hotel']=htl.at[rnum_htl,'wkn_web']
                df.at[i+j,'hotel_name']=htl.at[rnum_htl,'wkn_name']
            else:
                df.at[i+j,'hotel']=[htl.at[rnum_htl,'wk_web'],htl.at[rnum_htl,'wkn_web']]
                df.at[i+j,'hotel_name']=[htl.at[rnum_htl,'wk_name'],htl.at[rnum_htl,'wkn_name']]
        else:
            df.at[i+j,'hotel']='End'
            df.at[i+j,'hotel_name']='End'
            df.at[i+j,'hotel_cost']=0
    i+=6
#Calculate the transportation cost, meal cost and cumulative cost in each city
i=0
for row in range(0,int(df.shape[0]/6)):
    for j in range(0,6):
        if (j!=5):
            df.at[i+j,'meal_cost']=df.at[i+j,'hotel_cost']*0.5
            df.at[i+j,'transportation_cost']=df.at[i+j,'flight_cost']*0.1
        else:
            df.at[i+j,'meal_cost']=0
            df.at[i+j,'transportation_cost']=0
        if(j==0):
            df.at[i+j,'cumulative_cost']=df.at[i+j,'hotel_cost']+df.at[i+j,'flight_cost']+df.at[i+j,'meal_cost']+df.at[i+j,'transportation_cost']
        else:
            df.at[i+j,'cumulative_cost']=df.at[i+j-1,'cumulative_cost']+df.at[i+j,'hotel_cost']+df.at[i+j,'flight_cost']+df.at[i+j,'meal_cost']+df.at[i+j,'transportation_cost']
    i+=6
df

,city,persons,start_date,wk,wkn,hotel,hotel_name,flight,hotel_cost,meal_cost,flight_cost,transportation_cost,cumulative_cost
0,NY(JFK),1,1,2,1,"[EKT_wk_name, EKT_wkn_name]","[Hilton New York JFK Airport, SAVE 12%\nHilton...",Start,782,391,0,0,1173
1,ABQ,2,4,1,1,"[CheapOAir_wk_name, CheapOAir_wkn_name]","[Doubletree By Hilton Albuquerque, Doubletree ...",Expedia_wk,476,238,120,12,2019
2,CLE,3,6,0,3,EKT_wkn_name,Hilton Garden Inn Cleveland Airport,CheapOAir_wkn,1431,715.5,367.6,36.76,4569.86
3,MIA,4,9,3,0,CheapOAir_wk_name,Hilton Miami Airport,Kayak_wkn,1908,954,162,16.2,7610.06
4,SEA,5,12,0,4,EKT_wkn_name,Hilton Seattle Airport & Conference Center,CheapOAir_wkn,2780,1390,827.2,82.72,12690
5,"[NY(JFK), ABQ, CLE, MIA]",0,16,0,1,End,End,"[[Kayak_wkn], [CheapOAir_wkn], [Kayak_wkn], [K...",0,0,623.8,0,13313.8
6,NY(JFK),1,1,2,1,"[EKT_wk_name, EKT_wkn_name]","[Hilton New York JFK Airport, SAVE 12%\nHilton...",Start,782,391,0,0,1173
7,ABQ,2,4,1,1,"[CheapOAir_wk_name, CheapOAir_wkn_name]","[Doubletree By Hilton Albuquerque, Doubletree ...",Expedia_wk,476,238,120,12,2019
8,CLE,3,6,0,3,EKT_wkn_name,Hilton Garden Inn Cleveland Airport,CheapOAir_wkn,1431,715.5,367.6,36.76,4569.86
9,SEA,4,9,3,0,EKT_wk_name,Hilton Seattle Airport & Conference Center,Kayak_wkn,1908,954,456,45.6,7933.46


In [6]:
#Sum up the cumulative result for each iteration
df_iter=pd.DataFrame()

for i in range(0,int(df.shape[0]/6)):
    iteration=df.loc[i*6,'city']+','+df.loc[i*6+1,'city']+','+df.loc[i*6+2,'city']+','+df.loc[i*6+3,'city']+','+df.loc[i*6+4,'city']
    df_iter.at[i,'iteration']=iteration
    df_iter.at[i,'cost']=df.at[i*6+5,'cumulative_cost']
df_iter=df_iter.sort_values(by='cost')
df_iter

,iteration,cost
15,"NY(JFK),MIA,CLE,SEA,ABQ",11404.94
21,"NY(JFK),SEA,CLE,MIA,ABQ",11514.42
23,"NY(JFK),SEA,MIA,CLE,ABQ",11625.06
17,"NY(JFK),MIA,SEA,CLE,ABQ",11814.38
18,"NY(JFK),SEA,ABQ,CLE,MIA",11988.16
9,"NY(JFK),CLE,MIA,SEA,ABQ",11990.98
14,"NY(JFK),MIA,CLE,ABQ,SEA",12032.84
19,"NY(JFK),SEA,ABQ,MIA,CLE",12121.16
16,"NY(JFK),MIA,SEA,ABQ,CLE",12157.50
11,"NY(JFK),CLE,SEA,MIA,ABQ",12237.62


In [158]:
#Save the file as .csv
df.to_csv(r'C:\Users\Messi\Desktop\Classes\DSCC\Project01\rawdata.csv')
df_iter.to_csv(r'C:\Users\Messi\Desktop\Classes\DSCC\Project01\summary.csv')